In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, utils
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
#set random seed
torch.manual_seed(11)

#load MNIST training and test datasets

train_dataset = datasets.MNIST(root='data',
                              train=True,
                              download=True,
                              transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))]))
test_dataset = datasets.MNIST(root='data',
                              train=False,
                              download=True,
                              transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))]))

In [3]:
#split training dataset into training and validation sets
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, 
                                                           [50000, 10000],
                                                           generator=torch.Generator().manual_seed(11))

#create filters for datasets so only 5s and 8s are included
train_filter = [idx for idx, sample in enumerate(train_dataset) if sample[1] in [5,8]]
val_filter = [idx for idx, sample in enumerate(val_dataset) if sample[1] in [5,8]]


#create dataloaders using filtered training and test datasets
train_dataloader = torch.utils.data.DataLoader(torch.utils.data.Subset(train_dataset, train_filter),
                                              batch_size = 64)
val_dataloader = torch.utils.data.DataLoader(torch.utils.data.Subset(val_dataset, val_filter),
                                              batch_size = 64)

In [4]:
#check to make sure dataloaders are properly filtered

train_labels = next(iter(train_dataloader))[1]
print('training labels: ', train_labels)

val_labels = next(iter(val_dataloader))[1]
print('val labels: ', val_labels)

training labels:  tensor([8, 8, 5, 5, 8, 8, 5, 5, 8, 5, 5, 8, 5, 5, 5, 5, 5, 5, 8, 5, 5, 8, 5, 8,
        8, 5, 5, 8, 5, 8, 8, 8, 5, 5, 8, 8, 5, 5, 5, 5, 8, 8, 8, 8, 5, 8, 5, 8,
        8, 8, 8, 8, 5, 5, 5, 5, 8, 8, 8, 8, 5, 8, 5, 5])
val labels:  tensor([5, 5, 5, 5, 5, 8, 8, 8, 5, 5, 8, 8, 5, 5, 8, 5, 8, 8, 5, 8, 5, 8, 8, 5,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5, 5, 8, 5, 5, 5, 8, 8, 8, 8, 8, 5, 8, 5,
        8, 8, 8, 5, 5, 8, 5, 5, 8, 5, 5, 8, 8, 5, 5, 8])


In [5]:
#define network for image processing
#outputs are in the form of (log class probabilities, hidden features)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # 10 channels in first convolution layer
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # 20 channels in second conv. layer
        self.fc1 = nn.Linear(320, 10) # 10 hidden units in first fully-connected layer
        self.fc2 = nn.Linear(10, 2) # 2 output units

    def forward(self, x):

        # first convolutional layer
        h_conv1 = self.conv1(x)
        h_conv1 = F.relu(h_conv1)
        h_conv1_pool = F.max_pool2d(h_conv1, 2)

        # second convolutional layer
        h_conv2 = self.conv2(h_conv1_pool)
        h_conv2 = F.relu(h_conv2)
        h_conv2_pool = F.max_pool2d(h_conv2, 2)

        # fully-connected layer
        h_fc1 = h_conv2_pool.view(-1, 320)
        h_fc1 = self.fc1(h_fc1)
        h_fc1 = F.relu(h_fc1)
        
        # classifier output
        output = self.fc2(h_fc1)
        output = F.log_softmax(output,dim=1)
        return output, h_fc1

In [6]:
#define training loop

def train_one_epoch():
    running_loss = 0.
    last_loss = 0.
    
    for i, batch in enumerate(train_dataloader):
        data = batch[0]
        target = batch[1]
        
        optimizer.zero_grad()
        output = model(data)[0]
        
        #relabeling target values so 5-->0 and 8-->1
        new_target = torch.tensor([0 if label==5 else 1 for label in target])
        
        loss = criterion(output, new_target)
        #print('loss: ', loss)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%10 == 9:
            last_loss = running_loss/10
            print('batch {} loss: {}'.format(i+1, last_loss))
            running_loss = 0.
        
            
    return last_loss

In [7]:
#set training parameters
model = CNN()
criterion = torch.nn.NLLLoss()
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_epochs = 5

best_val_loss = 10e6

#train model
for epoch in range(num_epochs):
    print('EPOCH {}'.format(epoch+1))
    
    model.train(True)
    avg_loss = train_one_epoch()
    
    #check validation loss
    model.train(False)
    running_val_loss = 0.
    for i, val_data in enumerate(val_dataloader):
        val_inputs, val_target = val_data
        val_outputs = model(val_inputs)[0]
        
        #relabeling target values so 5-->0 and 8-->1
        new_val_target = torch.tensor([0 if label==5 else 1 for label in val_target])
        
        val_loss = criterion(val_outputs, new_val_target)
        running_val_loss += val_loss
    
    avg_val_loss = running_val_loss / (i+1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_val_loss))
    
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'cnn')

EPOCH 1
batch 10 loss: 0.6660315454006195
batch 20 loss: 0.55852070748806
batch 30 loss: 0.3826448380947113
batch 40 loss: 0.20611062049865722
batch 50 loss: 0.14634240940213203
batch 60 loss: 0.06813690923154354
batch 70 loss: 0.09970612227916717
batch 80 loss: 0.06790848411619663
batch 90 loss: 0.04834919441491366
batch 100 loss: 0.050461013056337836
batch 110 loss: 0.037664445023983716
batch 120 loss: 0.04826828418299556
batch 130 loss: 0.04373614629730582
batch 140 loss: 0.04619812034070492
LOSS train 0.04619812034070492 valid 0.04880813509225845
EPOCH 2
batch 10 loss: 0.03526162952184677
batch 20 loss: 0.048914133291691544
batch 30 loss: 0.03928437726572156
batch 40 loss: 0.041596430353820324
batch 50 loss: 0.04085503877140582
batch 60 loss: 0.014235061220824718
batch 70 loss: 0.045776332076638934
batch 80 loss: 0.03707020878791809
batch 90 loss: 0.020973848761059344
batch 100 loss: 0.01804286534897983
batch 110 loss: 0.019134345883503556
batch 120 loss: 0.015225190436467528
batch

In [8]:
#extract hidden features from training dataset

#initialize results array
hidden_features_results = np.empty([len(train_dataloader)*64, 11])

#iterate through training dataloader and collect hidden features
for i, batch in enumerate(train_dataloader):
    data = batch[0]
    target = batch[1]
    
    log_probs, hidden_features = model(data)
    
    batch_results = np.empty([len(target), 11])
    
    batch_results[:,0] = target
    batch_results[:,1:] = hidden_features.detach().numpy()
    
    hidden_features_results[i*64:(i*64)+len(target)] = batch_results
    
#store results in dataframe
cols = ['label',
        'feature_1',
        'feature_2',
        'feature_3',
        'feature_4',
        'feature_5',
        'feature_6',
        'feature_7',
        'feature_8',
        'feature_9',
        'feature_10']

hidden_features_df = pd.DataFrame(data = hidden_features_results, columns = cols)
hidden_features_df['label'] = hidden_features_df['label'].astype(int)
#hidden_features_df = hidden_features_df[np.logical_or(hidden_features_df['label']==5, hidden_features_df['label']==8)]
print(hidden_features_df.head())

#save dataframe as csv
hidden_features_df.to_csv('hidden_features_df.csv')

   label  feature_1  feature_2  feature_3  feature_4  feature_5  feature_6  \
0      8        0.0  11.246423        0.0   7.995950   9.410959  19.745337   
1      8        0.0  13.312637        0.0  10.743755  11.378502  18.715256   
2      5        0.0   0.000000        0.0   0.000000   0.000000  28.648830   
3      5        0.0   0.000000        0.0   0.000000   0.000000  26.991035   
4      8        0.0   5.038842        0.0   5.323320   5.368689  22.266983   

   feature_7  feature_8  feature_9  feature_10  
0   9.010060   8.527996  10.110795   19.589340  
1  11.810072  11.065614  12.528616   18.708977  
2   0.000000   0.000000   0.000000   30.524609  
3   0.000000   0.000000   0.000000   29.041384  
4   4.806798   4.124915   4.903548   23.818264  
